In [90]:
!rm -f /tmp/*.json
!rm -f /tmp/*.pb

In [91]:
import os
import json
import subprocess
import pandas as pd
 
pwd0 = "/home/jovyan/work/fabric-samples/"
pwd = "/home/jovyan/work/fabric-samples/test-network"

os.environ['PATH'] = "/home/jovyan/work/fabric-samples/bin:" + str(os.environ.get('PATH')) 
os.environ['CORE_PEER_TLS_ENABLED'] = "true"
os.environ['CORE_PEER_LOCALMSPID'] = "Org1MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE'] = pwd + "/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt"
os.environ['CORE_PEER_MSPCONFIGPATH'] = pwd + "/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp"
os.environ['CORE_PEER_ADDRESS'] = "peer0.org1.example.com:7051"
os.environ['FABRIC_CFG_PATH'] = pwd0 + "/config/"

addr0 = 'peer0.org1.example.com:7050'
orderer = 'orderer.example.com'
ordererpem = pwd + '/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem'
addr1 = 'peer0.org1.example.com:7051'
crt1 = pwd + '/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt'
addr2 = 'peer0.org2.example.com:9051'
crt2 = pwd + '/organizations/peerOrganizations/org2.example.com/peers/peer0.org2.example.com/tls/ca.crt' 

msp1 = pwd + '/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp'
msp2 = pwd + '/organizations/peerOrganizations/org2.example.com/users/Admin@org2.example.com/msp'

In [92]:
os.environ['ORDERER_IP']="10.15.128.97"
os.environ['ORDERER_HOSTNAME']="orderer.example.com" 
os.environ['CHANNEL_NAME']="mychannel"
os.environ['ORG_ID']="OrgSally"

In [93]:
os.environ['CORE_PEER_LOCALMSPID']="Org1MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE']=crt1
os.environ['CORE_PEER_MSPCONFIGPATH']=msp1
os.environ['ORDERER_CA']=ordererpem

os.environ['CORE_PEER_ADDRESS'] = "peer0.org1.example.com:7051"

In [94]:
#update /etc/hosts
#!more /etc/hosts
#echo $ORDERER_IP orderer.example.com >> /etc/hosts1
!peer channel fetch config /tmp/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA > /dev/null
!configtxlator proto_decode --input /tmp/config_block.pb --type common.Block | jq .data.data[0].payload.data.config > /tmp/config.json    

2020-12-07 15:53:27.467 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2020-12-07 15:53:27.471 UTC [cli.common] readBlock -> INFO 002 Received block: 16
2020-12-07 15:53:27.471 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 2
2020-12-07 15:53:27.476 UTC [cli.common] readBlock -> INFO 004 Received block: 2


In [95]:
# Upload CA cert
os.environ['SHARED_FS_HOST']="10.15.136.41:30080"
os.environ['SHARED_FS_USER']="scray"
os.environ['SHARED_FS_PW']="scray"
#!apk add curl

In [96]:
os.environ['NEW_ORG_NAME']="OrgSally"

!curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/ca
!curl --user $SHARED_FS_USER:$SHARED_FS_PW -T /home/jovyan/work/fabric-samples/test-network/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem

# Get configuration of new peer
#curl --user 'scray:scray' http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > new_member_org.json
!curl --user $SHARED_FS_USER:$SHARED_FS_PW  http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > /tmp/new_member_org.json   

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="http://10.15.136.41:30080/ca/">here</a>.</p>
</body></html>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11435  100 11435    0     0  1861k      0 --:--:-- --:--:-- --:--:-- 1861k


In [97]:
# Add org3 data to existing config
!jq -s ".[0] "*" {\"channel_group\":{\"groups\":{\"Application\":{\"groups\": {\"${NEW_ORG_NAME}MSP\":.[1]}}}}}" /tmp/config.json /tmp/new_member_org.json > /tmp/modified_config.json

!configtxlator proto_encode --input /tmp/config.json --type common.Config --output /tmp/config.pb
!configtxlator proto_encode --input /tmp/modified_config.json --type common.Config --output /tmp/modified_config.pb
!configtxlator compute_update --channel_id $CHANNEL_NAME --original /tmp/config.pb --updated /tmp/modified_config.pb --output /tmp/org3_update.pb
!configtxlator proto_decode --input /tmp/org3_update.pb --type common.ConfigUpdate | jq . > /tmp/org3_update.json
!echo '{"payload":{"header":{"channel_header":{"channel_id":"'$CHANNEL_NAME'", "type":2}},"data":{"config_update":'$(cat /tmp/org3_update.json)'}}}' | jq . > /tmp/org3_update_in_envelope.json
!configtxlator proto_encode --input /tmp/org3_update_in_envelope.json --type common.Envelope --output /tmp/org3_update_in_envelope.pb

In [98]:
!peer channel signconfigtx -f /tmp/org3_update_in_envelope.pb

2020-12-07 15:53:56.892 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized


In [99]:
os.environ['CORE_PEER_LOCALMSPID']="Org2MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE']=crt2
os.environ['CORE_PEER_MSPCONFIGPATH']=msp2
os.environ['ORDERER_CA']=ordererpem
os.environ['CORE_PEER_ADDRESS'] = addr2

!peer channel update -f /tmp/org3_update_in_envelope.pb -c $CHANNEL_NAME -o orderer.example.com:7050 --tls --cafile $ORDERER_CA

2020-12-07 15:54:03.469 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2020-12-07 15:54:03.494 UTC [channelCmd] update -> INFO 002 Successfully submitted channel update


In [100]:
print(os.environ.get('CORE_PEER_MSPCONFIGPATH'))

#!ls /home/jovyan/work/fabric-samples/test-network/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp
#/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp
!ls /home/jovyan/work/fabric-samples/test-network/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp

/home/jovyan/work/fabric-samples/test-network/organizations/peerOrganizations/org2.example.com/users/Admin@org2.example.com/msp
admincerts  cacerts  config.yaml  keystore  signcerts  tlscacerts


In [ ]:
ORDERER_IP=$1
CHANNEL_NAME=$2 #mychannel
NEW_ORG_NAME=$3

export CORE_PEER_LOCALMSPID="Org1MSP"
export CORE_PEER_TLS_ROOTCERT_FILE=/opt/gopath/src/github.com/hyperledger/fabric/peer/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt
export CORE_PEER_MSPCONFIGPATH=/opt/gopath/src/github.com/hyperledger/fabric/peer/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp
export CORE_PEER_ADDRESS=peer0.org1.example.com:7051
export ORDERER_CA=/opt/gopath/src/github.com/hyperledger/fabric/peer/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem

echo $ORDERER_IP orderer.example.com >> /etc/hosts

# Export existing channel configuration
peer channel fetch config config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA
configtxlator proto_decode --input config_block.pb --type common.Block | jq .data.data[0].payload.data.config > config.json

# Upload CA cert
SHARED_FS_HOST=10.15.136.41:30080
SHARED_FS_USER=scray
SHARED_FS_PW=scray
apk add curl
curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/ca
curl --user $SHARED_FS_USER:$SHARED_FS_PW -T /opt/gopath/src/github.com/hyperledger/fabric/peer/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem

# Get configuration of new peer
curl --user 'scray:scray' http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > new_member_org.json


# Add org3 data to existing config
jq -s ".[0] "*" {\"channel_group\":{\"groups\":{\"Application\":{\"groups\": {\"${NEW_ORG_NAME}MSP\":.[1]}}}}}" config.json ./new_member_org.json > modified_config.json

configtxlator proto_encode --input config.json --type common.Config --output config.pb
configtxlator proto_encode --input modified_config.json --type common.Config --output modified_config.pb
configtxlator compute_update --channel_id $CHANNEL_NAME --original config.pb --updated modified_config.pb --output org3_update.pb
configtxlator proto_decode --input org3_update.pb --type common.ConfigUpdate | jq . > org3_update.json
echo '{"payload":{"header":{"channel_header":{"channel_id":"'$CHANNEL_NAME'", "type":2}},"data":{"config_update":'$(cat org3_update.json)'}}}' | jq . > org3_update_in_envelope.json
configtxlator proto_encode --input org3_update_in_envelope.json --type common.Envelope --output org3_update_in_envelope.pb

# Sign update by first admin
peer channel signconfigtx -f org3_update_in_envelope.pb

# Sign by second admin
export CORE_PEER_LOCALMSPID="Org2MSP"
export CORE_PEER_TLS_ROOTCERT_FILE=/opt/gopath/src/github.com/hyperledger/fabric/peer/organizations/peerOrganizations/org2.example.com/peers/peer0.org2.example.com/tls/ca.crt
export CORE_PEER_MSPCONFIGPATH=/opt/gopath/src/github.com/hyperledger/fabric/peer/organizations/peerOrganizations/org2.example.com/users/Admin@org2.example.com/msp
export CORE_PEER_ADDRESS=peer0.org2.example.com:9051
peer channel update -f org3_update_in_envelope.pb -c $CHANNEL_NAME -o orderer.example.com:7050 --tls --cafile $ORDERER_CA


In [60]:
!peer channel fetch config /tmp/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA > /dev/null  

2020-12-07 14:39:31.365 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2020-12-07 14:39:31.372 UTC [cli.common] readBlock -> INFO 002 Received block: 82
2020-12-07 14:39:31.372 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 82
2020-12-07 14:39:31.377 UTC [cli.common] readBlock -> INFO 004 Received block: 82


In [58]:
#peer channel fetch config config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA
import sys    
def channelFetch(channelName,ordererpem):
    #rint(channelName,ordererpem)
    try:    
        output = subprocess.check_output(['peer', 'channel','fetch', 'config', 'config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', 
                                               '--cafile', ordererpem], stderr=subprocess.STDOUT, shell=True, timeout=3,universal_newlines=True)
        #output = subprocess.check_output(['peer', 'channel','fetch', 'config', 'config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', 'cafile', ordererpem], stderr=subprocess.STDOUT, shell=True, timeout=3,universal_newlines=True)
        #output = subprocess.check_output(['peer', 'channel','fetch','newest', 'config', '/tmp/config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', '--cafile', ordererpem], stderr=subprocess.STDOUT)
        
    #xcept Exception as e:
    #   print('exception' + str(e))
    #   return None
    except subprocess.CalledProcessError as exc:
        print("Status : FAIL", exc.returncode, exc.output)
    else:
        print("Output: \n{}\n".format(output))
    
#channelFetch('mychannel',ordererpem)    

#!cd /home/jovyan/work/
#rm -f config_block.pb
channelFetch('mychannel',ordererpem)


Output: 
Usage:
  peer [command]

Available Commands:
  chaincode   Operate a chaincode: install|instantiate|invoke|package|query|signpackage|upgrade|list.
  channel     Operate a channel: create|fetch|join|list|update|signconfigtx|getinfo.
  help        Help about any command
  lifecycle   Perform _lifecycle operations
  node        Operate a peer node: start|reset|rollback|pause|resume|rebuild-dbs|upgrade-dbs.
  version     Print fabric peer version.

Flags:
  -h, --help   help for peer

Use "peer [command] --help" for more information about a command.


